In [1]:

cd ..

/Users/sgemma.sun/Documents/data101/airbnb-ml


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import s3fs
import pyarrow.parquet as pq

s3 = s3fs.S3FileSystem()

filePath = 's3://airbnb-barcelona/valid/currentDate=2020-03-10'
airbnb_df = pq.ParquetDataset(filePath, filesystem=s3).read_pandas().to_pandas()

airbnb = airbnb_df.drop(columns=[
    'rowId',
    'id',
    'host_location',
    'host_neighbourhood',
    'street',
    'neighbourhood',
    'neighbourhood_cleansed',
    'market',
    'license'
])

print(airbnb_df.shape)
print(airbnb_df.dtypes.value_counts())

(20981, 66)
object     25
float64    22
int32      14
bool        5
dtype: int64


In [5]:
cat_df = airbnb.select_dtypes(include=['object']).copy()
cat_df.head(1)

,host_since,host_response_time,host_is_superhost,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,zipcode,property_type,room_type,bed_type,amenities,calendar_updated,first_review,last_review,cancellation_policy
0,2010-01-19,within an hour,False,"['email', 'phone', 'reviews', 'jumio', 'government_id']",True,True,Eixample,08025,Apartment,Entire home/apt,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelchair accessible"",Kitchen,Elevator,""Free street parking"",""Buzzer/wireless intercom"",Heating,""Family/kid friendly"",Washer,Essentials,Shampoo,Hangers,""Hair dryer"",Iron,""Laptop friendly workspace"",Crib,""Hot water"",""Host greets you"",""Paid parking on premises""}",today,2013-05-27,2019-11-01,strict_14_with_grace_period


In [11]:
import src.transform as trans
cat_df = trans.encode_boolean_to_float(cat_df, "host_is_superhost")
cat_df = trans.encode_boolean_to_float(cat_df, "host_has_profile_pic")
cat_df = trans.encode_boolean_to_float(cat_df, "host_identity_verified")
cat_df = trans.drop_rows_occurs_less_than(cat_df, "zipcode", 7)

In [13]:
import functools
def encode_category_dic(dataframe):
    def h(dica, columnName):
        labels = dataframe[columnName].astype('category').cat.categories.tolist()
        encode_dic = {columnName: {k: v for k, v in zip(labels, list(range(1, len(labels)+1)))}}
        return dict(dica, **encode_dic)
    return h


def foldleft(func, acc, xs):
    return functools.reduce(func, xs, acc)

In [18]:
category_encoder = encode_category_dic(airbnb)
category_columns = [
    "neighbourhood_group_cleansed",
    "property_type",
    "room_type",
    "bed_type",
    "encode_category_dic",
]
category_dic = foldleft(category_encoder, {}, category_columns)
category_dic
# cat_df["neighbourhood_group_cleansed"].value_counts()

KeyError: 'encode_category_dic'

In [ ]:
numeric_df = airbnb_df.select_dtypes(include=['float64', 'int32']).copy()

In [ ]:
replace_map = foldleft(airbnb_replace, {}, category_columns)
replace_map

In [ ]:
cat_airbnb = airbnb_drop.replace(mapp)

# fillna

In [ ]:
airbnb_drop.isnull().sum()